##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2: inicio rápido para principiantes

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar notebook</a>   </td>
</table>

En esta breve introducción se usa [Keras](https://www.tensorflow.org/guide/keras/overview) para:

1. Cargar un conjunto de datos predeterminado.
2. Crear un modelo de aprendizaje automático de red neural que clasifique las imágenes.
3. Entrenar la red neural.
4. Evaluar la exactitud del modelo.

Este tutorial es un notebook de [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb). Los programas de Python se ejecutan directamente en tu navegador —una gran manera de aprender y utilizar TensorFlow. Para poder seguir este tutorial, presione el botón que se encuentra en la parte superior de esta página y ejecute este notebook en Google Colab.

1. En Colab, conéctese al tiempo de ejecución de Python: en la parte superior derecha de la barra de menús, seleccione *CONNECT*.
2. Para ejecutar todo el código en el notebook, seleccione **Runtime** &gt; **Run all**. Para ejecutar las celdas de a una, coloque el mouse sobre cada celda y seleccione el ícono para **ejecutar la celda**.

![Ícono de Ejecutar celda](https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/quickstart/images/beginner/run_cell_icon.png?raw=1)

## Preparación de TensorFlow

Para empezar, importe TensorFlow a su programa:

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.0


Si está siguiendo los pasos en su propio entorno de desarrollo y no en [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), consulte esta [guía de instalación](https://www.tensorflow.org/install) para configurar TensorFlow para el desarrollo.

Nota: controle haber actualizado a la última versión de `pip` para instalar el paquete de TensorFlow 2 en caso de que use su propio entorno de desarrollo. Para más detalles, consulte la [guía de instalación](https://www.tensorflow.org/install).

## Carga de un conjunto de datos

Cargue y prepare [conjunto de datos MNIST](http://yann.lecun.com/exdb/mnist/). Los valores de los pixeles de las imágenes van de 0 a 255. Escale esos valores a un rango de 0 a 1 dividiendo los valores por `255.0`. De este modo, también se convierten los datos de muestra de los enteros a números de punto flotante:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Creación de un modelo de aprendizaje automático

Cree un modelo `tf.keras.Sequential`:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) es útil para apilar capas donde cada una tiene un [tensor](https://www.tensorflow.org/guide/tensor) de entrada y uno de salida. Las capas son funciones con una estructura matemática desconocida que se puede reutilizar y que tiene variables entrenables. La mayoría de los modelos TensorFlow están compuestos por capas. Este modelo usa las capas [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) y [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout).

Para cada ejemplo, el modelo devuelve un vector de [logits](https://developers.google.com/machine-learning/glossary#logits) o puntajes de [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) (registro de probabilidades) por cada clase.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.32516694,  0.10137546, -1.339407  , -0.06653149,  0.3763236 ,
         0.6608368 , -0.40918964,  0.1052368 ,  0.15436256, -0.15151629]],
      dtype=float32)

La función `tf.nn.softmax` convierte estas funciones logits en *probabilidades* para cada clase:

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.07067493, 0.10827056, 0.02563224, 0.09153545, 0.1425341 ,
        0.18944421, 0.06497926, 0.10868943, 0.11416221, 0.08407771]],
      dtype=float32)

Nota: es posible aplicar la función `tf.nn.softmax` en la función de activación para la última capa de la red. Si bien esto puede hacer que la salida del modelo se interprete más directamente, este enfoque no se recomienda ya que es imposible proporcionar un cálculo de pérdida numéricamente estable y exacto para todos los modelos con salida softmax.

Defina la función de pérdida para el entrenamiento con `losses.SparseCategoricalCrossentropy`:

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

La función de pérdida toma un vector de valores verdaderos de base y un vector de logits y devuelve una pérdida escalar para cada ejemplo. Esta pérdida es igual a la probabilidad de registro negativa de la clase verdadera: La pérdida es cero si el modelo está seguro de la clase correcta.

El modelo sin entrenar arroja probabilidades cercanas al lo aleatorio (1/10 para cada clase), entonces, la pérdida inicial debería estar cerca de `-tf.math.log(1/10) ~= 2.3`.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

1.6636606

Antes de empezar el entrenamiento, configure y compile el modelo con Keras `Model.compile`. Configure la clase del [`optimizador`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) como `adam`, establezca `loss` para la función `loss_fn` que definió antes y especifique una métrica a evaluar para el modelo, mediante la determinación del parámetro `metrics` para `accuracy`.

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Entrenamiento y evaluación del modelo

Use el método `Model.fit` para ajustar los parámetros del modelo y minimizar la pérdida:

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8617 - loss: 0.4688
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9561 - loss: 0.1514
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9667 - loss: 0.1095
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9745 - loss: 0.0840
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9765 - loss: 0.0747


El método `Model.evaluate` controla el desempeño del modelo, por lo general con un [conjunto de evaluación](https://developers.google.com/machine-learning/glossary#validation-set) o un [conjunto de prueba](https://developers.google.com/machine-learning/glossary#test-set).

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - 2ms/step - accuracy: 0.9739 - loss: 0.0761


[0.07609912008047104, 0.9739000201225281]

El clasificador de imágenes ahora está entrenado para proporcionar ~98% de exactitud en este conjunto de datos. Para más información, lea los [tutoriales de TensorFlow](https://www.tensorflow.org/tutorials/).

Si desea que su modelo devuelva una probabilidad, puede empaquetar el modelo entrenado y adjuntarle el softmax:

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[6.8604598e-09, 8.9592792e-11, 4.1947733e-06, 5.0097203e-04,
        2.0385263e-10, 1.4281011e-06, 4.6286458e-15, 9.9945432e-01,
        9.9722854e-08, 3.8932467e-05],
       [1.3365174e-10, 1.9433870e-04, 9.9980444e-01, 4.1552707e-07,
        6.1376185e-15, 6.8506228e-07, 8.5791765e-09, 4.5956235e-13,
        6.3055573e-08, 2.5926119e-12],
       [1.5324528e-07, 9.9916101e-01, 4.1411797e-05, 3.3730298e-06,
        1.1694097e-05, 1.4299078e-05, 1.9518855e-05, 7.1925798e-04,
        2.8422633e-05, 8.4951870e-07],
       [9.9949539e-01, 2.2908091e-09, 2.2007202e-04, 4.3981751e-07,
        8.5802267e-06, 8.6851230e-07, 1.8441038e-04, 4.7649726e-05,
        1.4422307e-07, 4.2449472e-05],
       [3.3450935e-06, 1.3265968e-09, 2.8895668e-06, 2.5296742e-07,
        9.9508554e-01, 3.0584652e-06, 3.6902720e-06, 9.9734942e-05,
        1.9996842e-06, 4.7994759e-03]], dtype=float32)>

## Conclusión

¡Felicitaciones! Ha entrenado un modelo de entrenamiento automático con un conjunto de datos predeterminado usando la API [Keras](https://www.tensorflow.org/guide/keras/overview).

Para acceder a más ejemplos sobre el uso de Keras, consulte los [tutoriales](https://www.tensorflow.org/tutorials/keras/). Para más información sobre cómo crear modelos con Keras, lea las [guías](https://www.tensorflow.org/guide/keras). Si quiere aprender más sobre cómo cargar y preparar los datos, mire los tutoriales sobre [carga de datos de imágenes](https://www.tensorflow.org/tutorials/load_data/images) o [carga de datos CSV](https://www.tensorflow.org/tutorials/load_data/csv).
